In [2]:
#Import the library
import pandas as pd
import os
import numpy as np
import seaborn as sns

In [4]:
#Load the data
train = pd.read_csv("train_Wc8LBpr.csv")

In [5]:
#Have a glimpse of the data
train.head()

,Trip_ID,Trip_Distance,Type_of_Cab,Customer_Since_Months,Life_Style_Index,Confidence_Life_Style_Index,Destination_Type,Customer_Rating,Cancellation_Last_1Month,Var1,Var2,Var3,Gender,Surge_Pricing_Type
0,T0005689460,6.77,B,1.0,2.42769,A,A,3.90500,0,40.0,46,60,Female,2
1,T0005689461,29.47,B,10.0,2.78245,B,A,3.45000,0,38.0,56,78,Male,2
2,T0005689464,41.58,NaN,10.0,NaN,NaN,E,3.50125,2,NaN,56,77,Male,2
3,T0005689465,61.56,C,10.0,NaN,NaN,A,3.45375,0,NaN,52,74,Male,3
4,T0005689467,54.95,C,10.0,3.03453,B,A,3.40250,4,51.0,49,102,Male,2


In [6]:
train.shape


(131662, 14)

In [7]:
train.dtypes


Trip_ID                         object
Trip_Distance                  float64
Type_of_Cab                     object
Customer_Since_Months          float64
Life_Style_Index               float64
Confidence_Life_Style_Index     object
Destination_Type                object
Customer_Rating                float64
Cancellation_Last_1Month         int64
Var1                           float64
Var2                             int64
Var3                             int64
Gender                          object
Surge_Pricing_Type               int64
dtype: object

In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
import xgboost as xgb
import numpy as np
import pandas as pd

train = pd.read_csv("train_Wc8LBpr.csv")
test = pd.read_csv("test_VsU9xXK.csv")

train['Gender'] = train['Gender'].replace(to_replace = {'Male': 0, 'Female': 1}) #XGBoost works on only numeric vectors 
test['Gender'] = test['Gender'].replace(to_replace = {'Male': 0, 'Female': 1})

type_of_cab = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
train['Type_of_Cab'] = train['Type_of_Cab'].replace(to_replace = type_of_cab) #XGBoost works on only numeric vectors 
test['Type_of_Cab'] = test['Type_of_Cab'].replace(to_replace = type_of_cab)


confi_index = {'A': 0, 'B': 1, 'C': 2}
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace(to_replace = confi_index)
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace(to_replace = confi_index)

train['Surge_Pricing_Type'] = train['Surge_Pricing_Type'] - 1

X_train = train.copy()
X_test = test.copy()

from sklearn.preprocessing import LabelEncoder
print("Label Encoding...")
for f in ['Destination_Type']:
    lbl = LabelEncoder()
    lbl.fit(list(X_train[f].values) + list(X_test[f].values)) #transforming values in Destination type using label encoder 
    X_train[f] = lbl.transform(list(X_train[f].values))
    X_test[f] = lbl.transform(list(X_test[f].values))

features = np.setdiff1d(train.columns, ['Trip_ID', 'Surge_Pricing_Type']) # returns set difference of the two arrays that have been 
                                                                          #passed as the arguments. Feature extraction, all columns except first and last       
params = {"objective": "multi:softmax","booster": "gbtree", "nthread": 4, "silent": 1,
                "eta": 0.08, "max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.7,
                "min_child_weight": 1, "num_class": 3,
                "seed": 2016, "tree_method": "exact"}
dtrain = xgb.DMatrix(X_train[features], X_train['Surge_Pricing_Type'], missing=np.nan)
dtest = xgb.DMatrix(X_test[features], missing=np.nan)

nrounds = 260
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)
test_preds = bst.predict(dtest)


Label Encoding...
[0]	train-merror:0.50001
[20]	train-merror:0.30118
[40]	train-merror:0.29519
[60]	train-merror:0.29221
[80]	train-merror:0.28902
[100]	train-merror:0.28630
[120]	train-merror:0.28383
[140]	train-merror:0.28157
[160]	train-merror:0.27959
[180]	train-merror:0.27805
[200]	train-merror:0.27618
[220]	train-merror:0.27437
[240]	train-merror:0.27291
[259]	train-merror:0.27119


In [7]:
submit = pd.DataFrame({'Trip_ID': test['Trip_ID'], 'Surge_Pricing_Type': test_preds+1})
submit.to_csv("XGB.csv", index=False)